In [66]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report

In [2]:
## transformers==0.45.0
## tokenizers== 0.20.3

base_model = "meta-llama/Llama-3.2-3B-Instruct"
fine_tuned_model = "marcoorasch/llama-3.2-3B-instruct-hatespeech-offensive-classification"

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch


# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model)

model = AutoModelForCausalLM.from_pretrained(
        fine_tuned_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        offload_buffers=True
)

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [13]:
!pip install transformers==4.45.0

  Obtaining dependency information for transformers==4.45.0 from https://files.pythonhosted.org/packages/31/6d/09a57274b45a411e6cc62122a032dd4f7cc6fb9e5323d4376ccc1d74bc56/transformers-4.45.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for tokenizers<0.21,>=0.20 from https://files.pythonhosted.org/packages/50/f6/2841de926bc4118af996eaf0bdf0ea5b012245044766ffc0347e6c968e63/tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.9 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successf

In [3]:
import tokenizers
import transformers

print("Tokenizers version:", tokenizers.__version__)
print("Transformers version:", transformers.__version__)


Tokenizers version: 0.20.3
Transformers version: 4.45.0


In [4]:
text = "You are a pig!"


In [38]:
prompt = f"""Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.
text: {text}
label: """.strip()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2,
    temperature=0.1,
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.float16,
    device_map="gpu"
)

outputs = pipe(prompt, max_new_tokens=2, do_sample=True, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
print(outputs[0]["generated_text"].split("label: ")[-1].strip())

Offensive


In [48]:
def generate_test_prompt(data_point,colname):
    return f"""
            Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.
            text: {data_point[colname]}
            label: """.strip()

In [49]:
def predict(test, model, tokenizer, colname):
    y_pred = []
    labels = ["Hate", "Offensive", "Normal"]
    
    for i in tqdm(range(len(test))):
        prompt = test.iloc[i][colname]
        # pipe = pipeline(task="text-generation", 
        #                 model=model, 
        #                 tokenizer=tokenizer, 
        #                 max_new_tokens=2, 
        #                 temperature=0.1)
        
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        
        # Determine the predicted category
        for label in labels:
            if label.lower() in answer.lower():
                y_pred.append(label)
                break
        else:
            y_pred.append("none")
    
    return y_pred


### TweetOffensive

In [8]:
from datasets import load_dataset

ds = load_dataset("cardiffnlp/tweet_eval", "offensive")

README.md:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/93.7k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/122k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/860 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1324 [00:00<?, ? examples/s]

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11916
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 860
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1324
    })
})

In [12]:
tweetoff = pd.concat([ds['train'].to_pandas(),ds['test'].to_pandas(),ds['validation'].to_pandas()])
tweetoff.head()

,text,label
0,@user Bono... who cares. Soon people will unde...,0
1,@user Eight years the republicans denied obama...,1
2,@user Get him some line help. He is gonna be j...,0
3,@user @user She is great. Hi Fiona!,0
4,@user She has become a parody unto herself? Sh...,1


In [14]:
prompts = pd.DataFrame(tweetoff.apply(generate_test_prompt, axis=1), columns=["text"])
prompts.head()

,text
0,"Classify the text into Hatespeech, Offensive, ..."
1,"Classify the text into Hatespeech, Offensive, ..."
2,"Classify the text into Hatespeech, Offensive, ..."
3,"Classify the text into Hatespeech, Offensive, ..."
4,"Classify the text into Hatespeech, Offensive, ..."


In [25]:
prompts.iloc[2,0]

'Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n            text: @user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time\n            label:'

In [ ]:
pred = predict(prompts, model, tokenizer)
pred[:5]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation. 8.79it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation. 9.76it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.10.80it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.11.90it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.11.21it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.10.35it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting

In [37]:
pred[:5]

['Normal', 'Normal', 'Normal', 'Normal', 'Offensive']

In [62]:
tweetoff['preiction'] = pred
tweetoff['prediction'].value_counts()

prediction
Normal       11151
Offensive     2940
Hate             9
Name: count, dtype: int64

In [59]:
tweetoff['pred_label'] = tweetoff['preiction'].map({'Normal':0,'Hate':0,'Offensive':1})
tweetoff.head()

,text,label,preiction,pred_label
0,@user Bono... who cares. Soon people will understand that they gain nothing from following a phony celebrity. Become a Leader of your people instead or help and support your fellow countrymen.,0,Normal,0
1,@user Eight years the republicans denied obama’s picks. Breitbarters outrage is as phony as their fake president.,1,Normal,0
2,@user Get him some line help. He is gonna be just fine. As the game went on you could see him progressing more with his reads. He brought what has been missing. The deep ball presence. Now he just needs a little more time,0,Normal,0
3,@user @user She is great. Hi Fiona!,0,Normal,0
4,@user She has become a parody unto herself? She has certainly taken some heat for being such an....well idiot. Could be optic too Who know with Liberals They're all optics. No substance,1,Offensive,1


In [68]:
print(classification_report(tweetoff['label'],tweetoff['pred_label'],target_names=['Not Offensive','Offensive']))

               precision    recall  f1-score   support

Not Offensive       0.75      0.89      0.82      9460
    Offensive       0.64      0.41      0.50      4640

     accuracy                           0.73     14100
    macro avg       0.70      0.65      0.66     14100
 weighted avg       0.72      0.73      0.71     14100



In [69]:
print(tweetoff['label'].value_counts())
print(tweetoff['pred_label'].value_counts())

label
0    9460
1    4640
Name: count, dtype: int64
pred_label
0    11160
1     2940
Name: count, dtype: int64


In [63]:
tweetoff.to_csv("TweetOffensive_llama.csv",index=False)

### OffensiveLang

In [50]:
offlang = pd.read_csv("OffensiveLang/OffensiveLang.csv")
offlang.head()

,Text,Category,Target Group,Final Annotation,OpenAI_Annotation,Annotator1,Annotator2,Annotator3
0,Your recent performances are lacking the spark that once defined your career.,Occupation,Actor,Offensive,Not Offensive,Not Offensive,Offensive,Offensive
1,I heard your agent is struggling to find you quality roles. Any truth to that?,Occupation,Actor,Offensive,Offensive,Offensive,Offensive,Offensive
2,Your character lacked depth; it felt like a caricature rather than a real person.,Occupation,Actor,Offensive,Not Offensive,Offensive,Offensive,Offensive
3,You were overshadowed by the set design; it was more memorable than your acting.,Occupation,Actor,Offensive,Offensive,Offensive,Offensive,Not Offensive
4,You're not as talented as [another actor] in that role.,Occupation,Actor,Offensive,Offensive,Offensive,Not Offensive,Offensive


In [54]:
offlang_prompts = pd.DataFrame(offlang.apply(generate_test_prompt, axis=1, args=("Text",)), columns=["Text"])
offlang_prompts.head()

,Text
0,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: Your recent performances are lacking the spark that once defined your career.\n label:"
1,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: I heard your agent is struggling to find you quality roles. Any truth to that?\n label:"
2,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: Your character lacked depth; it felt like a caricature rather than a real person.\n label:"
3,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: You were overshadowed by the set design; it was more memorable than your acting.\n label:"
4,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: You're not as talented as [another actor] in that role.\n label:"


In [34]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2,
    temperature=0.1,
    pad_token_id=tokenizer.eos_token_id,
    torch_dtype=torch.float16,
    device_map="gpu"
)

In [57]:
offlang_pred = predict(offlang_prompts, model, tokenizer,colname="Text")
offlang_pred[:5]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
100%|███████████████████████████████████████| 8270/8270 [10:00<00:00, 13.77it/s]


['Normal', 'Normal', 'Normal', 'Normal', 'Normal']

In [64]:
offlang['prediction'] = offlang_pred
offlang['pred_label'] = offlang['prediction'].apply(lambda x: 1 if x=='Offensive' else 0)
offlang.head()

,Text,Category,Target Group,Final Annotation,OpenAI_Annotation,Annotator1,Annotator2,Annotator3,prediction,pred_label
0,Your recent performances are lacking the spark that once defined your career.,Occupation,Actor,Offensive,Not Offensive,Not Offensive,Offensive,Offensive,Normal,0
1,I heard your agent is struggling to find you quality roles. Any truth to that?,Occupation,Actor,Offensive,Offensive,Offensive,Offensive,Offensive,Normal,0
2,Your character lacked depth; it felt like a caricature rather than a real person.,Occupation,Actor,Offensive,Not Offensive,Offensive,Offensive,Offensive,Normal,0
3,You were overshadowed by the set design; it was more memorable than your acting.,Occupation,Actor,Offensive,Offensive,Offensive,Offensive,Not Offensive,Normal,0
4,You're not as talented as [another actor] in that role.,Occupation,Actor,Offensive,Offensive,Offensive,Not Offensive,Offensive,Normal,0


In [71]:
offlang['true_label'] = offlang['Final Annotation'].apply(lambda x: 1 if x=="Offensive" else 0)

In [72]:
print(classification_report(offlang['true_label'],offlang['pred_label'],target_names=['Not Offensive','Offensive']))

               precision    recall  f1-score   support

Not Offensive       0.22      0.95      0.36      1748
    Offensive       0.88      0.09      0.16      6522

     accuracy                           0.27      8270
    macro avg       0.55      0.52      0.26      8270
 weighted avg       0.74      0.27      0.20      8270



In [65]:
offlang.to_csv("offlang_llama.csv",index=False)

### TDavidson

In [73]:
from datasets import load_dataset

ds = load_dataset("tdavidson/hate_speech_offensive")

In [76]:
td = ds['train'].to_pandas()
td.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [77]:
td_prompts = pd.DataFrame(td.apply(generate_test_prompt, axis=1, args=("tweet",)), columns=["tweet"])
td_prompts.head()

,tweet
0,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...\n label:"
1,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: !!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!\n label:"
2,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit\n label:"
3,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: !!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny\n label:"
4,"Classify the text into Hatespeech, Offensive, Normal and return the answer as the corresponding label.\n text: !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;\n label:"


In [78]:
td_pred = predict(td_prompts, model, tokenizer,colname="tweet")
td_pred[:5]

100%|█████████████████████████████████████| 24783/24783 [30:14<00:00, 13.66it/s]


['Normal', 'Offensive', 'Offensive', 'Offensive', 'Offensive']

In [79]:
td['prediction'] = td_pred
td['pred_label'] = td['prediction'].apply(lambda x: 1 if x=='Offensive' else 0)
td['true_label'] = td['class'].apply(lambda x: 1 if x==1 else 0)
print(classification_report(td['true_label'],td['pred_label'],target_names=['Not Offensive','Offensive']))

               precision    recall  f1-score   support

Not Offensive       0.88      0.65      0.74      5593
    Offensive       0.90      0.97      0.94     19190

     accuracy                           0.90     24783
    macro avg       0.89      0.81      0.84     24783
 weighted avg       0.90      0.90      0.89     24783



In [80]:
td.to_csv('TDavidson_llama.csv',index=False)